In [1]:
!pip install git+https://github.com/HKUDS/LightRAG.git

from lightrag import LightRAG
from lightrag.llm.openai import openai_complete_if_cache
from lightrag.kg.shared_storage import initialize_pipeline_status
from lightrag.utils import EmbeddingFunc, setup_logger

  Running command git clone --filter=blob:none --quiet https://github.com/HKUDS/LightRAG.git 'C:\Users\sckwo\AppData\Local\Temp\pip-req-build-zoptu7oh'

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Cloning https://github.com/HKUDS/LightRAG.git to c:\users\sckwo\appdata\local\temp\pip-req-build-zoptu7oh
  Resolved https://github.com/HKUDS/LightRAG.git to commit c87eb2cfcf096e2e421851a2679ddffe9a57d10b
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached aiohttp-3.12.15-cp312-cp312-win_amd64.whl.metadata (7.9 kB)
  Using cached configparser-7.2.0-py3-none-any.whl.metadata (5.5 kB)
  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached nano_vectordb-0.0.4.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached numpy-2.3.2-cp312-cp312-win_amd64.w

2025-09-08 11:21:50 - pipmaster.package_manager - INFO - Executing: C:\Users\sckwo\PycharmProjects\Graph-Builder\.venv\Scripts\python.exe -m pip install --upgrade openai
2025-09-08 11:22:32 - pipmaster.package_manager - INFO - Command succeeded: C:\Users\sckwo\PycharmProjects\Graph-Builder\.venv\Scripts\python.exe -m pip install --upgrade openai


In [ ]:
# Устанавливаем все нужные пакеты за один раз
!pip install spacy pymorphy3 tqdm nltk torch transformers openai

import codecs
import json
import os
import random
import re
import asyncio

import nest_asyncio
from typing import List

from tqdm.notebook import tqdm

import nltk
from nltk.stem.snowball import SnowballStemmer
import numpy as np
from openai import OpenAI
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

import codecs
import copy
import json
from typing import Dict, List, Tuple
import warnings

import spacy
from pymorphy3 import MorphAnalyzer
from pymorphy3.analyzer import Parse


In [2]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [3]:
dataset_dir = 'pages_txt'
print(f'os.path.isdir({dataset_dir}) = {os.path.isdir(dataset_dir)}')

os.path.isdir(pages_txt) = True


In [4]:
text_files = list(map(lambda it2: os.path.join(dataset_dir, it2), filter(lambda it1: it1.endswith('.txt'), os.listdir(dataset_dir))))

In [30]:
POS_DICT: Dict[str, str] = {
    'NOUN': 'NOUN',
    'PROPN': 'NOUN',
    'PRON': 'NPRO',
    'DET': 'NPRO',
    'ADJ': 'ADJF',
    'VERB': 'VERB',
    'AUX': 'PRCL',
    'PART': 'PRCL',
    'ADV': 'ADVB',
    'NUM': 'NUMR',
    'ADP': 'PREP',
    'CCONJ': 'CONJ',
    'SCONJ': 'CONJ',
    'INTJ': 'INTJ'
}

CASE_DICT: Dict[str, str] = {
    'Acc': 'accs',
    'Dat': 'datv',
    'Gen': 'gent',
    'Ins': 'ablt',
    'Loc': 'loct',
    'Nom': 'nomn',
    'Par': 'gen2',
    'Voc': 'voct'
}

NUMBER_DICT: Dict[str, str] = {
    'Sing': 'sing',
    'Plur': 'plur'
}


def initialize_abbreviation_subsystem(config_name: str) -> (
        Tuple)[Dict[str, Tuple[List[str], List[Tuple[str, str, str]]]], spacy.Language, MorphAnalyzer]:
    nlp = spacy.load('ru_core_news_sm')
    analyzer = MorphAnalyzer()
    with codecs.open(config_name, mode='r', encoding='utf-8', errors='ignore') as fp:
        data = json.load(fp)
    if not isinstance(data, dict):
        err_msg = f'The abbreviation config "{config_name}" contains a wrong information!'
        raise ValueError(err_msg)
    keys = list(data.keys())
    abbr = dict()
    for k in keys:
        if not isinstance(k, str):
            err_msg = f'The abbreviation config "{config_name}" contains a wrong information!'
            raise ValueError(err_msg)
        if not k.isalpha():
            err_msg = f'The abbreviation config "{config_name}" contains a wrong information!'
            raise ValueError(err_msg)
        if not isinstance(data[k], str):
            err_msg = f'The abbreviation config "{config_name}" contains a wrong information!'
            raise ValueError(err_msg)
        v = data[k].strip()
        if len(v) == 0:
            err_msg = f'The abbreviation config "{config_name}" contains a wrong information!'
            raise ValueError(err_msg)
        doc = nlp(v)
        tokens = []
        morpho_data = []
        for token in doc:
            tokens.append(token.text)
            if str(token.pos_) in POS_DICT:
                pos = POS_DICT[str(token.pos_)]
            else:
                pos = str(token.pos_)
            case = token.morph.get('Case')
            if len(case) > 0:
                if len(case[0]) > 0:
                    case = CASE_DICT[str(case[0])]
                else:
                    case = ''
            else:
                case = ''
            number = token.morph.get('Number')
            if len(number) > 0:
                if len(number[0]) > 0:
                    number = NUMBER_DICT[str(number[0])]
                else:
                    number = ''
            else:
                number = ''
            morpho_data.append((pos, case, number))
        abbr[k.lower()] = (tokens, morpho_data)
        del doc, tokens, morpho_data
    return abbr, nlp, analyzer


def tokenize_and_analyze_morphology(text: str, nlp: spacy.Language) -> (
        Tuple)[List[str], List[Tuple[int, int]], List[Tuple[str, str, str]]]:
    doc = nlp(text)
    all_tokens = []
    all_bounds = []
    all_morpho = []
    for token in doc:
        all_tokens.append(token.text)
        all_bounds.append((token.idx, token.idx + len(token)))
        pos = POS_DICT.get(str(token.pos_), str(token.pos_))
        case = token.morph.get('Case')
        if len(case) > 0:
            if len(case[0]) > 0:
                case = CASE_DICT[str(case[0])]
                if case == 'accs':
                    case = 'loct'
            else:
                case = ''
        else:
            case = ''
        number = token.morph.get('Number')
        if len(number) > 0:
            if len(number[0]) > 0:
                number = NUMBER_DICT[str(number[0])]
            else:
                number = ''
        else:
            number = ''
        all_morpho.append((pos, case, number))
    del doc
    return all_tokens, all_bounds, all_morpho


def find_main_token(phrase_tokens: List[str], morpho: List[Tuple[str, str, str]]) -> int:
    found_idx = -1
    n = len(phrase_tokens)
    if n != len(morpho):
        err_msg = (f'Number of tokens does not equal to number of morphological items! '
                   f'{phrase_tokens} != {morpho}')
        raise ValueError(err_msg)
    for idx in range(n):
        if (morpho[idx][0] in {'NOUN', 'NPRO'}) and (morpho[idx][1] == 'nomn'):
            found_idx = idx
    return found_idx


def find_form(token: str, morpho: Tuple[str, str, str], analyzer: MorphAnalyzer) -> Parse:
    variants = analyzer.parse(token)
    best = variants[0]
    for it in variants[1:]:
        if it.tag.POS == morpho[0] and (morpho[2] in {'sing', 'plur'} and it.tag.number == morpho[2]):
            best = it
    return best


def inflect_phrase(phrase_tokens: List[str], morpho: List[Tuple[str, str, str]], inflector: MorphAnalyzer,
                   target_case: str, target_number: str) -> str:
    main_token_idx = find_main_token(phrase_tokens, morpho)
    if main_token_idx < 0:
        warnings.warn(f'The text "{" ".join(phrase_tokens)}" cannot be inflected!')
        return ' '.join(phrase_tokens)
    inflected_tokens = []
    for idx, val in enumerate(phrase_tokens):
        if idx <= main_token_idx:
            # Исключаем пустые значения
            target_grammemes = set()
            if target_case:
                target_grammemes.add(target_case)
            if target_number:
                target_grammemes.add(target_number)

            inflection = find_form(val, morpho[idx], inflector).inflect(target_grammemes)
            if inflection is None:
                inflected_tokens.append(val)
            else:
                inflected_tokens.append(inflection.word)
        else:
            inflected_tokens.append(val)
    if len(inflected_tokens) == 0:
        return ''
    if inflected_tokens[0][0].islower():
        inflected_tokens[0] = inflected_tokens[0][0].upper() + inflected_tokens[0][1:]
    return ' '.join(inflected_tokens)


def replace_abbreviations(old_text: str, abbreviation_config: Dict[str, Tuple[List[str], List[Tuple[str, str, str]]]],
                          nlp: spacy.Language, morph: MorphAnalyzer) -> str:
    tokens, bounds, morpho_data = tokenize_and_analyze_morphology(old_text, nlp)
    new_text = copy.copy(old_text)
    for idx, (token, (pos, case, number)) in enumerate(zip(tokens, morpho_data)):
        if token.lower() in abbreviation_config:
            case = case if case in CASE_DICT.values() else ''
            number = number if number in NUMBER_DICT.values() else ''
            new_token = inflect_phrase(
                abbreviation_config[token.lower()][0],
                abbreviation_config[token.lower()][1],
                morph,
                case,
                number
            )
            new_text = new_text[:bounds[idx][0]] + new_token + new_text[bounds[idx][1]:]
            if len(new_token) != len(token):
                bounds[idx] = (
                    bounds[idx][0],
                    bounds[idx][1] + len(new_token) - len(token)
                )
                for other_idx in range(idx + 1, len(bounds)):
                    bounds[other_idx] = (
                        bounds[other_idx][0] + len(new_token) - len(token),
                        bounds[other_idx][1] + len(new_token) - len(token)
                    )
    return new_text


In [31]:
import os
import sys

# python -m spacy download ru_core_news_sm
config_fname = os.path.join('full_abbreviations_updated.json')
config, spacy, pymorphy = initialize_abbreviation_subsystem(config_fname)

def convert(text: str) -> str:
    return replace_abbreviations(text, config, spacy, pymorphy)


In [32]:
text_data = []
for cur_fname in text_files:
    with codecs.open(cur_fname, mode='r', encoding='utf-8', errors='ignore') as fp:
        new_text = '\n'.join(list(map(
            lambda it3: ' '.join(it3.replace('\r', ' ').split()),
            filter(
                lambda it2: len(it2) > 0,
                map(
                    lambda it1: it1.strip(),
                    fp.readlines()
                )
            )
        ))).strip()
    if len(new_text) > 0:
        text_data.append(convert(new_text))
    del new_text

C:\Users\sckwo\AppData\Local\Temp\ipykernel_6384\3072762628.py:160: UserWarning: The text "механико - математический факультет" cannot be inflected!
  warnings.warn(f'The text "{" ".join(phrase_tokens)}" cannot be inflected!')
C:\Users\sckwo\AppData\Local\Temp\ipykernel_6384\3072762628.py:160: UserWarning: The text "Open Researcher and Contributor ID" cannot be inflected!
  warnings.warn(f'The text "{" ".join(phrase_tokens)}" cannot be inflected!')
C:\Users\sckwo\AppData\Local\Temp\ipykernel_6384\3072762628.py:160: UserWarning: The text "федеральный государственный образовательный стандарт" cannot be inflected!
  warnings.warn(f'The text "{" ".join(phrase_tokens)}" cannot be inflected!')


In [36]:
print(f'Number of documents is {len(text_data)}.')
print(f'3 random documents:')
for it in random.sample(text_data, 3):
    print('\n' + it)

Number of documents is 473.
3 random documents:

Инструкция Программное обеспечение рассылке о сверке кадровых данных
Письмо с уведомлением о сверке кадровых данных получают пользователи, которые работают с документом Проект штатного расписания конкретной кафедры, и начальники Учебного - методического объединения факультета/института – Институтам математики в письме сообщается о проблемах Программное обеспечение всем кафедрам. В письме указан только номер задачи, конкретные проблемы можно увидеть только открыв саму задачу в 1С:Учебный процесс.
Если вам на почту пришло письмо с уведомлением, что необходимо произвести сверку кадровых данных, нужно сделать следующее:
1. Необходимо зайти в 1С:Учебный процесс в раздел «Текущие дела» - Мои задачи:
2. Если у вас нет такого блока на «Рабочем столе» в системе 1С: Учебный процесс, то его можно добавить. Для этого зайдите в «Главное меню» (кнопка в виде стрелки вниз на верхней кнопочной панели) – Вид – Настройка рабочего стола.
3. Далее в открывш

In [37]:
from tqdm.auto import tqdm
special_tokens = dict()
re_for_special_tokens = re.compile(r'\[\w+?\]')
for cur_text in tqdm(text_data):
    start_pos = 0
    search_res = re_for_special_tokens.search(cur_text[start_pos:])
    while search_res is not None:
        token_start = start_pos + search_res.start()
        token_end = start_pos + search_res.end()
        new_special_token = cur_text[token_start:token_end] 
        special_tokens[new_special_token] = special_tokens.get(new_special_token, 0) + 1
        start_pos = token_end
        search_res = re_for_special_tokens.search(cur_text[start_pos:])
special_token_keys = sorted(list(special_tokens.keys()), key=lambda it: (-special_tokens[it], it))
print(f'There are {len(special_tokens)} special tokens. They are:')
for it in special_token_keys:
    print('{0:>20}: {1:>6}'.format(it, special_tokens[it]))

100%|██████████| 473/473 [00:00<00:00, 152385.42it/s]

There are 2 special tokens. They are:
             [CACHE]:      2
          [MASSMAIL]:      1


In [38]:
setup_logger("lightrag", level="INFO")

0


In [39]:
WORKING_DIR = 'prepared_it_new'
if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

IndexError: list index out of range

In [9]:
VLLM_API_KEY = ''
VLLM_BASE_URL = 'everest.nsu.ru:9111'
os.environ['OPENAI_API_KEY'] = VLLM_API_KEY

In [8]:
LLM_NAME = 'RuadaptQwen3-32B-Instruct'
TEMPERATURE = 0.3
QUERY_MAX_TOKENS = 8000

In [24]:
LOCAL_EMBEDDER_DIMENSION = 768
LOCAL_EMBEDDER_MAX_TOKENS = 4096
LOCAL_EMBEDDER_NAME = '/workspace/data/models/gte-multilingual-base'
print(f'os.path.isdir({LOCAL_EMBEDDER_NAME}) = {os.path.isdir(LOCAL_EMBEDDER_NAME)}')

In [6]:
ABBREVIATIONS_FNAME = 'full_abbreviations_updated.json'
print(f'os.path.isfile({ABBREVIATIONS_FNAME}) = {os.path.isfile(ABBREVIATIONS_FNAME)}')

In [5]:
TEMPLATE_FOR_ABBREVIATION_EXPLAINING = '''Отредактируйте, пожалуйста, текст заданного документа так, чтобы этот документ стал более простым и понятным для обычных людей от юных старшеклассников до пожилых мужчин и женщин. При этом не надо, пожалуйста, применять markdown или иной вид гипертекста. Главное, на что вам надо обратить внимание и по возможности исправить - это логика изложения и понятность формулировок документа. Ничего не объясняйте и не комментируйте своё решение, просто перепишите текст документа.

Обратите внимание, что документ анонимизирован, то есть все именованные сущности заменены специальными словами-масками в квадратных скобках (например, вместо текста "Иван Иванович Иванов любит горчицу" вы встретите текст "[NAME] любит горчицу"). Полный список специальных слов-масок приведён здесь: {special_masks}. Не изменяйте этих слов, пожалуйста, а оставляйте как есть.

Также исправьте грамматические ошибки в тексте документа, если они там есть. Кроме того, если вы обнаружите аббревиатуры в тексте этого документа, то замените все обнаруженные аббревиатуры их корректными расшифровками, сохранив морфологическую и синтаксическую согласованность. Вот здесь вы можете ознакомиться с JSON-словарём, описывающим возможные аббревиатуры и их расшифровки:

```json
{abbreviations_dict}
```

Далее приведён текст документа, нуждающийся в возможном улучшении:

```text
{text_of_document}
```'''

os.path.isdir(C:\Users\sckwo\models\gte-multilingual-base) = True


In [4]:
async def llm_model_func(
    prompt, system_prompt=None, history_messages=[], keyword_extraction=False, **kwargs
) -> str:
    return await openai_complete_if_cache(
        LLM_NAME,
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        api_key=VLLM_API_KEY,
        base_url=VLLM_BASE_URL,
        temperature=TEMPERATURE,
        **kwargs
    )

os.path.isfile(full_abbreviations_updated.json) = True


In [10]:
async def gte_hf_embed(texts: List[str], tokenizer, embed_model) -> np.ndarray:
    device = next(embed_model.parameters()).device
    encoded_texts = tokenizer(
        texts, return_tensors='pt', padding=True, truncation=True
    ).to(device)
    batch_dict = tokenizer(
        texts, return_tensors='pt',
        max_length=LOCAL_EMBEDDER_MAX_TOKENS, padding=True, truncation=True,
    ).to(device)
    with torch.no_grad():
        outputs = embed_model(**batch_dict)
        embeddings = F.normalize(
            outputs.last_hidden_state[:, 0][:LOCAL_EMBEDDER_DIMENSION],
            p=2, dim=1
        )
    if embeddings.dtype == torch.bfloat16:
        return embeddings.detach().to(torch.float32).cpu().numpy()
    else:
        return embeddings.detach().cpu().numpy()

In [11]:
def explain_abbreviations_with_llm(document: str, abbreviations: dict) -> str:
    snow_stemmer = SnowballStemmer(language='russian')
    filtered_abbreviations = dict()
    for cur_word in nltk.wordpunct_tokenize(document):
        if cur_word in abbreviations:
            filtered_abbreviations[cur_word] = abbreviations[cur_word]
        elif cur_word.lower() in abbreviations:
            filtered_abbreviations[cur_word] = abbreviations[cur_word.lower()]
        elif cur_word.upper() in abbreviations:
            filtered_abbreviations[cur_word] = abbreviations[cur_word.upper()]
        else:
            stem = snow_stemmer.stem(cur_word)
            if stem in abbreviations:
                filtered_abbreviations[cur_word] = abbreviations[stem]
            elif stem.lower() in abbreviations:
                filtered_abbreviations[cur_word] = abbreviations[stem.lower()]
            elif stem.upper() in abbreviations:
                filtered_abbreviations[cur_word] = abbreviations[stem.upper()]
    del snow_stemmer
    if len(filtered_abbreviations) == 0:
        return document
    user_prompt = TEMPLATE_FOR_ABBREVIATION_EXPLAINING.format(
        abbreviations_dict=filtered_abbreviations,
        text_of_document=document,
        special_masks=special_token_keys
    )
    messages = [{'role': 'user', 'content': user_prompt}]
    global client
    try:
        response = client.chat.completions.create(
            model=LLM_NAME,
            messages=messages,
            temperature=TEMPERATURE,
            n=1,
            max_tokens=QUERY_MAX_TOKENS
        )
        new_improved_document = response.choices[0].message.content
        del response
    except:
        new_improved_document = document
    del messages, user_prompt
    return new_improved_document

In [12]:
async def initialize_rag():
    emb_tokenizer = AutoTokenizer.from_pretrained(
        LOCAL_EMBEDDER_NAME
    )
    emb_model = AutoModel.from_pretrained(
        LOCAL_EMBEDDER_NAME,
        trust_remote_code=True
    )
    emb_model.eval()
    
    rag = LightRAG(
        working_dir=WORKING_DIR,
        llm_model_func=llm_model_func,
        cosine_better_than_threshold=0.1,
        embedding_func=EmbeddingFunc(
            embedding_dim=LOCAL_EMBEDDER_DIMENSION,
            max_token_size=LOCAL_EMBEDDER_MAX_TOKENS,
            func=lambda texts: gte_hf_embed(
                texts,
                tokenizer=emb_tokenizer,
                embed_model=emb_model
            )
        ),
        addon_params={'language': 'Russian'},
    )

    await rag.initialize_storages()
    await initialize_pipeline_status()

    return rag

In [13]:
client = OpenAI(
    api_key=VLLM_API_KEY,
    base_url=VLLM_BASE_URL,
)

In [14]:
MAX_NUMBER_OF_TEXTS = None

In [15]:
improved_texts_fname = os.path.join(WORKING_DIR, 'improved_texts.json')

if os.path.isfile(improved_texts_fname):
    with open(improved_texts_fname, mode='r', encoding='utf-8') as fp:
        improved_texts = json.load(fp)

# data_iter = text_data if MAX_NUMBER_OF_TEXTS is None else text_data[:MAX_NUMBER_OF_TEXTS]
# # improved_texts = [convert(cur_text) for cur_text in tqdm(data_iter)]
# for cur_text in tqdm(data_iter):
#     improved_texts.append(convert(cur_text))
#
# with open(improved_texts_fname, mode='w', encoding='utf-8') as fp:
#     json.dump(improved_texts, fp, ensure_ascii=False, indent=4)


In [16]:
with codecs.open(improved_texts_fname, mode='w', encoding='utf-8') as fp:
    json.dump(fp=fp, obj=improved_texts, ensure_ascii=False, indent=4)

In [18]:
print(f'3 random examples of improved texts:')
if MAX_NUMBER_OF_TEXTS is None:
    selected_indices = random.sample(list(range(len(text_data))), 3)
else:
    selected_indices = random.sample(list(range(len(text_data[0:MAX_NUMBER_OF_TEXTS]))), 3)
for example_index in selected_indices:
    print('')
    print('BEFORE IMPROVING:')
    print(' '.join(text_data[example_index].split()))
    print('AFTER IMPROVING:')
    print(' '.join(improved_texts[example_index].split()))

In [47]:
from tqdm.auto import tqdm

nest_asyncio.apply()
rag = asyncio.run(initialize_rag())
if MAX_NUMBER_OF_TEXTS is None:
    for cur_text in tqdm(improved_texts):
        rag.insert(cur_text)
else:
    for cur_text in tqdm(improved_texts[0:MAX_NUMBER_OF_TEXTS]):
        rag.insert(cur_text)
rag.finalize_storages()

In [23]:
from tqdm.auto import tqdm

nest_asyncio.apply()
rag = asyncio.run(initialize_rag())
if MAX_NUMBER_OF_TEXTS is None:
    for cur_text in tqdm(improved_texts):
        rag.insert(cur_text)
else:
    for cur_text in tqdm(improved_texts[0:MAX_NUMBER_OF_TEXTS]):
        rag.insert(cur_text)
rag.finalize_storages()

Some weights of the model checkpoint at C:\Users\sckwo\models\gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO: [_] Created new empty graph fiel: prepared_it_new_2\graph_chunk_entity_relation.graphml
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': 'prepared_it_new_2\\vdb_entities.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': 'prepared_it_new_2\\vdb_relationships.json'} 0 data
INFO:nano-vectordb:Init

KeyboardInterrupt: 

INFO: limit_async: Creating 1 new workers
ERROR: OpenAI API Call Failed,
Model: openai/gpt-4.1,
Params: {'temperature': 0.3}, Got: Error code: 400 - {'error': {'message': 'User with this API key not found', 'code': 400}}
ERROR: limit_async: Error in decorated function: Error code: 400 - {'error': {'message': 'User with this API key not found', 'code': 400}}
ERROR: Failed to extract entities and relationships: Error code: 400 - {'error': {'message': 'User with this API key not found', 'code': 400}}
ERROR: Traceback (most recent call last):
  File "C:\Users\sckwo\PycharmProjects\light_rag_db\.venv\Lib\site-packages\lightrag\lightrag.py", line 1336, in process_document
    await entity_relation_task
  File "C:\Users\sckwo\AppData\Local\Programs\Python\Python312\Lib\asyncio\futures.py", line 287, in __await__
    yield self  # This tells Task to wait for completion.
    ^^^^^^^^^^
  File "C:\Users\sckwo\AppData\Local\Programs\Python\Python312\Lib\asyncio\tasks.py", line 375, in __wakeup
  